<a href="https://colab.research.google.com/github/Shezan57/LLM/blob/main/Training_embedding_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets

In [3]:
from datasets import load_dataset

In [4]:
# Load MNLI dataset from GLUE
train_dataset = load_dataset(
    'glue','mnli',split='train'
).select(range(80_000))
train_dataset = train_dataset.remove_columns('idx')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
train_dataset[2]

{'premise': 'One of our number will carry out your instructions minutely.',
 'hypothesis': 'A member of my team will execute your orders with immense precision.',
 'label': 0}

##Train Model

In [6]:
from sentence_transformers import SentenceTransformer

In [7]:
# Use a base model
embedding_model = SentenceTransformer('bert-base-uncased')

In [8]:
from sentence_transformers import losses

In [9]:
# Define the loss function. In softmax loss, we will also need to set
# the number of labels
train_loss = losses.SoftmaxLoss(
    model = embedding_model,
    sentence_embedding_dimension=embedding_model.get_sentence_embedding_dimension(),
    num_labels=3
)

In [12]:
# !pip install --upgrade sentence-transformers

In [14]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

In [17]:
from sentence_transformers import InputExample

In [18]:
# create an embedding similarity evaluator for STSB
val_sts = load_dataset('glue','stsb',split='validation')

# Create a list of InputExample objects
examples = [InputExample(texts=[s1, s2], label=score/5) for s1, s2, score in zip(val_sts['sentence1'], val_sts['sentence2'], val_sts['label'])]

# Initialize the evaluator with the list of InputExample objects
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(examples, main_similarity='cosine')

In [20]:
from sentence_transformers import SentenceTransformerTrainingArguments

In [26]:
#Define the training arguments
args = SentenceTransformerTrainingArguments(
    output_dir='base_embedding_model',
    num_train_epochs=3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100,
    report_to='none'
)


In [24]:
from sentence_transformers import SentenceTransformerTrainer

In [29]:
#train embedding model
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator
)
trainer.train()

dataset = dataset.select_columns(['hypothesis', 'entailment', 'contradiction'])


Step,Training Loss
100,1.069200
200,0.898700
300,0.842300
400,0.812400
500,0.792500
600,0.774500
700,0.756500
800,0.769700
900,0.753700
1000,0.728100


TrainOutput(global_step=3750, training_loss=0.553407162475586, metrics={'train_runtime': 1399.2582, 'train_samples_per_second': 171.519, 'train_steps_per_second': 2.68, 'total_flos': 0.0, 'train_loss': 0.553407162475586, 'epoch': 3.0})

In [30]:
# Evaluate our Traine model
evaluator(embedding_model)

{'pearson_cosine': 0.6210807792073179, 'spearman_cosine': 0.6646587095777491}